In [55]:
import os

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

os.environ["OPENAI_MODEL_NAME"] = (
    "gpt-3.5-turbo-0125"  # we need a larger context window for this project
)

In [56]:
from utils import get_google_api_client

google_api_client = get_google_api_client()

In [57]:
from langchain.embeddings import CacheBackedEmbeddings, SentenceTransformerEmbeddings
from langchain.storage import LocalFileStore

embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
store = LocalFileStore("L7_output/local_file_store")
embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embedding_model,
    document_embedding_cache=store,
)

/Users/deonna/LLMs for Devs/Week 4 - Agents/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [58]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

transcript_path = "kt_transcripts/663.txt"

loader = TextLoader(file_path=transcript_path)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunked_documents = text_splitter.split_documents(documents)

vector_store = FAISS.from_documents(
    documents=chunked_documents,
    embedding=embedder,
)

In [59]:
from crewai_tools import Tool


def search_kill_tony(query):
    docs = vector_store.similarity_search(query=query, k=5)
    return " ".join([doc.page_content for doc in docs])


rag_search_tool = Tool(
    name="Kill Tony Transcript RAG Searcher",
    func=search_kill_tony,
    description="Uses RAG to retrieve information from the transcript using a query",
)

In [69]:
from crewai import Agent

researcher = Agent(
    role="Mean Moments Researcher",
    goal="Find the meanest and most memorable moments from the Kill Tony episode transcript",
    backstory="""You are an expert at analyzing comedy podcasts to identify the meanest jokes, roasts, and interactions. 
    You have a keen sense of humor and can pick out the highlights that fans will love.""",
    tools=[rag_search_tool],
    verbose=True,
)

writer = Agent(
    role="Recap Writer",
    goal="Write an engaging and humorous recap of the best moments from the Kill Tony episode",
    backstory="""You are a talented comedy writer who can summarize the funniest parts of a podcast in a witty
    and entertaining way. Your recaps make readers feel like they experienced the best of the episode.""",
    verbose=True,
)

In [70]:
from crewai import Task


research_task = Task(
    description="""Read through the transcript of the Kill Tony episode and identify the top 5 meanest moments.""",
    expected_output="""Provide a brief description of each mean moment.""",
    output_file="L7_output/best_moments.md",
    agent=researcher,
)

writing_task = Task(
    description="""Take the top 5 mean moments identified by the Mean Moments Researcher and write an
    engaging and humorous recap. Focus on capturing the essence of what made each moment funny or memorable.""",
    expected_output="""A detailed, engaging recap of words of the best moments from the Kill Tony episode""",
    output_file="L7_output/recap.md",
    agent=writer,
)

In [71]:
from crewai import Crew

crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, writing_task],
    verbose=True,
)

result = crew.kickoff()

2024-06-05 20:36:39,052 - 8528120832 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Mean Moments Researcher
 [INFO]: == Starting Task: Read through the transcript of the Kill Tony episode and identify the top 5 meanest moments.


> Entering new CrewAgentExecutor chain...
I should start by using the Kill Tony Transcript RAG Searcher to find the top 5 meanest moments in the episode.

Action:
Kill Tony Transcript RAG Searcher
Action Input:
{
  "query": "top 5 meanest moments"
}
 

there uh 911 20
2 uh we're going to have a lot of fun
tonight you guys have both done the show
multiple times I have another special
announcement tonight
259 people
259 it could be a record I'm not exactly
sure I don't think we've ever actually
kept count but
259 in crowded in a bar across the
street right now hoping that one of the
young uh Buck uh producers runs over
there and yells their name then they get
held backstage until it is their time
I'm going to pre-pull a name now a very
lucky name and wow they are wrangling
that person we will get the show started
you

In [72]:
from IPython.display import Markdown, display

display(Markdown(research_task.output_file))

1. The host jokes about the number of people in the audience hoping for their name to be called, implying they are desperate for attention.
2. The soundboard operator is criticized for hitting the wrong buttons, with the host expressing frustration and making derogatory remarks.
3. Cam Patterson makes a joke about wanting to sleep with an old woman, followed by a joke about not understanding helping the homeless if they don't get houses.
4. A comedic skit involving a fake phone call between a father and son, where the father makes playful jabs at the son.
5. The host jokes about smoking a cigar, with a playful interaction about how to properly smoke it.

In [73]:
display(Markdown(writing_task.output_file))

As a talented comedy writer, I've taken the time to review the Kill Tony episode and identify the top 5 mean moments for an engaging and humorous recap:

1. The host jokingly mocks the eager audience members hoping to be called upon, implying their desperation for attention. It's like watching a pack of hungry wolves waiting for their chance to pounce on the spotlight.

2. The soundboard operator receives criticism for repeatedly hitting the wrong buttons, leading to the host's frustration and a series of derogatory remarks. It's a classic case of technology failing at the worst possible moment, turning a simple task into a comedy of errors.

3. Cam Patterson delivers a risque joke about wanting to sleep with an older woman, followed by a humorous take on the irony of helping the homeless without giving them actual homes. It's a blend of dark humor and social commentary that hits just the right spot.

4. A comedic skit unfolds with a fake phone call between a father and son, where the father playfully jabs at the son with witty remarks and banter. It's a hilarious exchange that showcases the dynamics of a playful yet loving relationship.

5. The host indulges in a cigar and engages in a playful interaction about the proper way to smoke it, leading to a lighthearted moment of comedic banter. It's a mix of sophistication and silliness that adds a touch of class to the episode.

With these top 5 mean moments highlighted, the Kill Tony episode is a rollercoaster of laughter and entertainment that keeps the audience engaged and amused throughout.